In [2]:
load("./curve_handler.sage")

#hex string to binary string
def sha1_bin(x):
    return format(ZZ(hashlib.sha1(bytes.fromhex(x)).hexdigest(), 16), '0160b')

def sha1_hex(x):
    return format(ZZ(sha1(x), 2), '0160x')

def increment_seed(seed, i = 1):
    g = ZZ(seed, 16).nbits()
    f = '0' + str(len(seed)) + 'X'
    return format(ZZ(Mod(ZZ(seed, 16) + i, 2^g)), f)

#http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.202.2977&rep=rep1&type=pdf, page 42
def generate_r(seed, p):
    t = p.nbits()
#     t = ceil(log(p,2))
#     originally floor
    s = floor((t-1)/160)
    h = t - 160 * s
#     print("t:",t, "s:", s, "h:", h)
    H = sha1_bin(seed)
    c0  = H[-h:]
    c0_modified = list(c0)
    c0_modified[0] = '0'
    W = [0] * (s + 1)
    W[0] =  ''.join(c0_modified)
    for i in [1..s]:
        input_i = increment_seed(seed, i)
        W[i] = sha1_bin(input_i)
    W_joint = ''.join(W)
    assert(len(W_joint) == t)
    r = 0
    for i in [1..t]:
        r += ZZ(W_joint[i-1]) * 2^(t-i)
    assert(r == ZZ(W_joint, 2))
    F = GF(p)
    return F(r)

def expected_r(p, a, b):
    F = GF(p)
    return F(a^3/b^2)

def verify_r(p, r, a, b):
    F = GF(p)
    return F(r) == expected_r(p, a, b)

def get_b_from_r(r, p, a = -3):
    F = GF(p)
    if F(a^3/r).is_square():
        return ZZ(F(a^3/r).sqrt())
    else:
        return None

def millerrabinliar(n,a):
    r = 0
    m = n-1
    while m % 2 == 0:
        r += 1
        m = m/2
    if Mod(a,n)^m == Mod(1,n):
        return True
    for i in range(r):
        if Mod(a,n)^(m*2^i) == Mod(-1,n):
            return True
    return False

def millerrabin(n):
    for a in prime_range(200):
        if not millerrabinliar(n, a):
            return False
    return True

# A2.2 in http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.202.2977&rep=rep1&type=pdf
def verify_near_primality(u, r_min, l_max = 255, MR = True):
    n = u
    h = 1
    for l in prime_range(l_max):
        while n % l == 0:
            n = ZZ(n / l)
            h = h * l
            if n < r_min:
                return False
    if MR and millerrabin(n):
        return True
    elif is_pseudoprime(n):
        return True #(h, n)
    return False

def verify_security(E, r_min_bits = 160, B = 20, MR = True, verbose = False):
    if verbose:
        print("Computing order")
    q = E.base_field().order()
    r_min = max(2^r_min_bits, 4 * sqrt(q))
    o = E.order()
    if verbose:
        print("Checking near-primality of", o)
    if not verify_near_primality(o, r_min, MR = MR):
        return False
    if verbose:
        print("Checking MOV")
    if embedding_degree(E, o) < B:
        return False
    if verbose:
        print("Checking Frob")
    if E.trace_of_frobenius() in [-1, 1]:
        return False
    return True

def generate_x962_curves(count, p, seed, increment = 1, fixed_a = True, verify = True, r_min_bits = 160, save_to_json = True, fname = 'curves_json/x962-sim/curves.json', MR = True):
    assert fixed_a
    a = -3
    
    bits = p.nbits()
    sim_curves = {}
    sim_curves["name"] = "x962_sim_" + str(bits)
    sim_curves["desc"] = "simulated curves generated according to the X9.62 standard using the initial seed " + str(seed)
    sim_curves["curves"] = []
    
    rel_seed = 0
    for i in [1..count]:
        rel_seed += increment * i
        if i%1000 == 0:
            print(count, "curves tried")
        seed_i = increment_seed(seed, increment * i)
        r = generate_r(seed_i, p)
        b = get_b_from_r(r, p)
        if b == None:
            continue
        E = EllipticCurve(GF(p), [a,b])
        if verify_security(E, r_min_bits, MR = MR):
            sim_curve = {
              "name": "x962_sim_" + str(bits) + "_rel_seed_" + str(rel_seed),
              "category": sim_curves["name"],
              "desc": "",
              "field": {
                "type": "Prime",
                "p": str(hex(p)),
                "bits": bits,
              },
              "form": "Weierstrass",
              "params": {
                "a": str(hex(-3)),
                "b": str(hex(b))
              },
              "generator": None,
              "order": E.order(),
              "cofactor": None,
              "characteristics": None,
              "seed": seed_i
            }
            sim_curves["curves"].append(sim_curve)
            
    if save_to_json:
        save_into_json(sim_curves, fname)
    return sim_curves

In [4]:
load("./curve_handler.sage")
curve_db = import_curve_db('curves_json')
curves = custom_curves(curve_db)

In [3]:
for c in curves:
    if "Ed" in c.form or "Mont" in c.form:
#         if c.field['type'] == "Binary":
#             print(c.name)
#             print(c.order * c.cofactor == c.EC.order())
            print(c.generator.order() == c.order)
#         print(c.generator * c.order == c.EC(0))
# curve_db.keys()

True
True
True
True


In [4]:
#bitlens, primes and corresponding seeds, case a=-3 (curves r1, prime fields only)
x962_params = []
x962_params.append((112, 0xDB7C2ABF62E35E668076BEAD208B, '00F50B028E4D696E676875615175290472783FB1'))
x962_params.append((128, 0xFFFFFFFDFFFFFFFFFFFFFFFFFFFFFFFF, '000E0D4D696E6768756151750CC03A4473D03679'))
x962_params.append((160, 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF7FFFFFFF, '1053CDE42C14D696E67687561517533BF3F83345'))
x962_params.append((192, 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFF, '3045AE6FC8422F64ED579528D38120EAE12196D5'))
x962_params.append((224, 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF000000000000000000000001, 'BD71344799D5C7FCDC45B59FA3B9AB8F6A948BC5'))
x962_params.append((256, 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF, 'C49D360886E704936A6678E1139D26B7819F7E90'))
x962_params.append((384, 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFF0000000000000000FFFFFFFF, 'A335926AA319A27A1D00896A6773A4827ACDAC73'))
x962_params.append((521, 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF, 'D09E8800291CB85396CC6717393284AAA0DA64BA'))

In [5]:
for bits, p, seed in x962_params[:1]:
    assert p.nbits() == bits
    assert len(seed) == 40
    fname = 'curves_json/x962-sim/curves' + str(bits) + '.json'
    print("Generating", bits, "-bit curves")
    generate_x962_curves(1, p, seed, -1, r_min_bits = bits-10, save_to_json = False, fname = fname)

Generating 112 -bit curves
